<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/czasy-joinow/colabs/Czasy_joinow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup sparka

##Setup sparka

In [1]:
!pip install pyspark py4j
!pip install -q findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar xf spark-3.3.1-bin-hadoop2.tgz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 47 kB/s 
     |████████████████████████████████| 200 kB 44.6 MB/s 
     |████████████████████████████████| 199 kB 64.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=5e4a431ccaa40943a0c9ac57ae7d3d93b2262264176569693a46a1a09e8dcb84
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
--2022-11-22 20:31:17--  ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
           => ‘spark-3.3.1-bin-hadoop2.tgz’
Resolving ftp.ps.pl (ftp.ps.pl)... 212.14.28.36, 2001:4c58:62:2::36
Connecting to ftp.ps.pl (ftp.ps.pl)|212.14.28.36|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/apache/spark/spark-3.3.1 ... done.
==>

In [2]:
import pyspark
import findspark
from pyspark.sql import SparkSession
import os

In [3]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
spark=SparkSession.builder.appName('Colab').getOrCreate()
spark

#Pobranie danych

##Import danych

In [4]:
!wget https://datasets.imdbws.com/name.basics.tsv.gz
!wget https://datasets.imdbws.com/title.akas.tsv.gz
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!wget https://datasets.imdbws.com/title.crew.tsv.gz
!wget https://datasets.imdbws.com/title.episode.tsv.gz
!wget https://datasets.imdbws.com/title.principals.tsv.gz
!wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2022-11-22 20:31:47--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.249.98.91, 13.249.98.73, 13.249.98.35, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.249.98.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236745635 (226M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz’

name.basics.tsv.gz  100%[===================>] 225.78M  87.4MB/s    in 2.6s    

2022-11-22 20:31:49 (87.4 MB/s) - ‘name.basics.tsv.gz’ saved [236745635/236745635]

--2022-11-22 20:31:49--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.249.98.91, 13.249.98.73, 13.249.98.35, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.249.98.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 288222028 (275M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz’

title.akas.tsv.gz   100%[===================>] 274.87M  31.2M

##Rozpakowanie danych

In [5]:
!gzip -dc /content/name.basics.tsv.gz > name.basics.csv
!gzip -dc /content/title.akas.tsv.gz > title.akas.csv
!gzip -dc /content/title.basics.tsv.gz > title.basic.csv
!gzip -dc /content/title.crew.tsv.gz > title.crew.csv
!gzip -dc /content/title.episode.tsv.gz > title.episode.csv
!gzip -dc /content/title.principals.tsv.gz > title.principals.csv
!gzip -dc /content/title.ratings.tsv.gz > title.ratings.csv

#Wczytanie danych

##Wczytajmy dane z rozpakowanych plików

In [6]:
df_name_basics=spark.read.option("header", "true").option("delimiter", "\t").csv('name.basics.csv' ) 
df_title_akas=spark.read.option("header","true").option("delimiter", "\t").csv('title.akas.csv')
df_title_basic=spark.read.option("header","true").option("delimiter", "\t").csv('title.basic.csv')
df_title_crew=spark.read.option("header","true").option("delimiter", "\t").csv('title.crew.csv')
df_title_episode=spark.read.option("header","true").option("delimiter","\t").csv('title.episode.csv')
df_title_principals=spark.read.option("header","true").option("delimiter","\t").csv('title.principals.csv')
df_title_ratings=spark.read.option("header","true").option("delimiter","\t").csv('title.ratings.csv')

##Wyświetlmy dane

In [7]:
df_name_basics.show(3)
df_title_akas.show(3)
df_title_basic.show(3)
df_title_crew.show(3)
df_title_episode.show(3)
df_title_principals.show(3)
df_title_ratings.show(3)

+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0072308,tt00531...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0117057,tt00383...|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0056404,tt00573...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 3 rows

+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|               title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|          Карменсіта|    

# Złączenie istotnych kolumn

## Wybierzmy następujące kolumny z tabel:
- df_title_basic:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - titleType (rodzaj tytułu, np. film, serial - można porównywać podobieństwo aktorów na podstawie tego w jakich typach produkcji grali)
  - originalTitle (tytuł produkcji - być może przyda się do analizy podobnych filmów)
  - isAdult (czy produkcja jest dla dorosłych - można wykorzystać do porównywania aktorów na podstawie tego czy grają głównie w produkcjach dla dorosłych)
  - genres (gatunki - może posłużyć do porównania aktorów na podstawie tego, że grali w produkcjach o podobnych gatunkach)

- df_title_principals:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - nconst (unikalny numer aktora - potrzebne do złączenia tabel)
  - category (kategoria pracy (actor/actress) - może posłużyć do porównania aktorów na podstawie takiej samej płci)
  - characters (postacie zagrane w produkcji - może posłużyć do porównania aktorów na podstawie podobnych granych postaci)

- df_name_basics:
  - nconst(unikalny numer aktora - potrzebne do złączenia tabel)
  - primaryName (imię i nazwisko aktora - potrzebne do wyświetlania aktora lub do odnajdywania go w bazie po imieniu i nazwisku)
  - birthYear (rok urodzenia aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - deathYear ( rok śmierci aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - knownForTitles (tytuły produkcji, z których znany jets aktor - może posłużyć do porównania aktorów na podstawie tych samych lub podobnych produkcji, z których są znani)

In [8]:
df_title_basic_selected = df_title_basic.select(["tconst", "titleType", "originalTitle", "isAdult", "genres"])
df_title_principals_selected = df_title_principals.select(["tconst", "nconst", "category", "characters"])
df_name_basics_selected = df_name_basics.select(["nconst", "primaryName", "birthYear", "deathYear", "knownForTitles"])

## Wybierzmy z tabeli df_title_principals_selected tylko aktorów i aktorki

In [9]:
#df_title_principals_selected = df_title_principals_selected.filter((df_title_principals_selected.category == "actor") | (df_title_principals_selected.category == "actress"))

# Złączmy tabele df_name_basics, df_title_principals i df_title_basic

## Sprawdźmy czasy joinowania dla różnych kombinacji tabel

Sprawdzimy najpierw jak długo zajmuje łączenie tabel bez usuniętych niepotrzebnych rekordów. Rozważmy różne kombinacje łączeń, gdzie tabele są łączone w następujący sposób:
* df_name_basics_selected i df_title_principals_selected, typ łączenia "inner",
* df_name_basics_selected i df_title_principals_selected, typ łączenia "right",
* dane wcześniej złączone i df_title_basic_selected, typ łączenia "inner",
* dane wcześniej złączone i df_title_basic_selected, typ łączenia "left",
* df_title_principals_selected i df_name_basics_selected, typ łączenia "inner",
* df_title_principals_selected i df_name_basics_selected, typ łączenia "left",
* df_title_basic_selected i df_title_principals_selected, typ łączenia "inner",
* df_title_basic_selected i df_title_principals_selected, typ łączenia "right",
* dane wcześniej złączone i df_name_basics_selected, typ łączenia "inner",
* dane wcześniej złączone i df_name_basics_selected, typ łączenia "right",
* df_title_principals_selected i df_title_basic_selected, typ łączenia "inner",
* df_title_principals_selected i df_title_basic_selected, typ łączenia "left".

In [10]:
%%time
data1 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data1.count(), len(data1.columns)))
data1 = data1.join(df_title_basic_selected, "tconst", "inner")
print("joined table size: ", (data1.count(), len(data1.columns)))

df_name_basics_selected and df_title_principals_selected joined table size:  (53163680, 8)
joined table size:  (53157572, 12)
CPU times: user 1.91 s, sys: 241 ms, total: 2.15 s
Wall time: 7min 30s


In [11]:
%%time
data2 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data2.count(), len(data2.columns)))
data2 = data2.join(df_title_basic_selected, "tconst", "left")
print("joined table size: ", (data2.count(), len(data2.columns)))

df_name_basics_selected and df_title_principals_selected joined table size:  (53163680, 8)
joined table size:  (53163680, 12)
CPU times: user 1.19 s, sys: 113 ms, total: 1.3 s
Wall time: 7min 14s


In [12]:
%%time
data3 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "right")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data3.count(), len(data3.columns)))
data3 = data3.join(df_title_basic_selected, "tconst", "inner")
print("joined table size: ", (data3.count(), len(data3.columns)))

df_name_basics_selected and df_title_principals_selected joined table size:  (53173247, 8)
joined table size:  (53167105, 12)
CPU times: user 1.22 s, sys: 153 ms, total: 1.37 s
Wall time: 7min 12s


In [13]:
%%time
data4 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "right")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data4.count(), len(data4.columns)))
data4 = data4.join(df_title_basic_selected, "tconst", "left")
print("joined table size: ", (data4.count(), len(data4.columns)))

df_name_basics_selected and df_title_principals_selected joined table size:  (53173247, 8)
joined table size:  (53173247, 12)
CPU times: user 1.25 s, sys: 125 ms, total: 1.38 s
Wall time: 7min 8s


In [14]:
%%time
data5 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "inner")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data5.count(), len(data5.columns)))
data5 = data5.join(df_title_basic_selected, "tconst", "inner")
print("joined table size: ", (data5.count(), len(data5.columns)))

df_title_principals_selected and df_name_basics_selected joined table size:  (53163680, 8)


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [15]:
%%time
data6 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "inner")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data6.count(), len(data6.columns)))
data6 = data6.join(df_title_basic_selected, "tconst", "left")
print("joined table size: ", (data6.count(), len(data6.columns)))

df_title_principals_selected and df_name_basics_selected joined table size:  (53163680, 8)
joined table size:  (53163680, 12)
CPU times: user 1.38 s, sys: 124 ms, total: 1.51 s
Wall time: 8min 1s


In [16]:
%%time
data7 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "left")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data7.count(), len(data7.columns)))
data7 = data7.join(df_title_basic_selected, "tconst", "inner")
print("joined table size: ", (data7.count(), len(data7.columns)))

df_title_principals_selected and df_name_basics_selected joined table size:  (53173247, 8)
joined table size:  (53167105, 12)
CPU times: user 1.23 s, sys: 120 ms, total: 1.35 s
Wall time: 7min 12s


In [17]:
%%time
data8 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "left")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data8.count(), len(data8.columns)))
data8 = data8.join(df_title_basic_selected, "tconst", "left")
print("joined table size: ", (data8.count(), len(data8.columns)))

df_title_principals_selected and df_name_basics_selected joined table size:  (53173247, 8)
joined table size:  (53173247, 12)
CPU times: user 1.22 s, sys: 105 ms, total: 1.32 s
Wall time: 7min 6s


In [18]:
%%time
data9 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "inner")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data9.count(), len(data9.columns)))
data9 = data9.join(df_name_basics_selected, "nconst", "inner")
print("joined table size: ", (data9.count(), len(data9.columns)))

df_title_basic_selected and df_title_principals_selected joined table size:  (53167105, 8)
joined table size:  (53157572, 12)
CPU times: user 1.07 s, sys: 90.6 ms, total: 1.16 s
Wall time: 6min 20s


In [19]:
%%time
data10 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "inner")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data10.count(), len(data10.columns)))
data10 = data10.join(df_name_basics_selected, "nconst", "left")
print("joined table size: ", (data10.count(), len(data10.columns)))

df_title_basic_selected and df_title_principals_selected joined table size:  (53167105, 8)
joined table size:  (53167105, 12)
CPU times: user 1.1 s, sys: 113 ms, total: 1.21 s
Wall time: 6min 41s


In [20]:
%%time
data11 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data11.count(), len(data11.columns)))
data11 = data11.join(df_name_basics_selected, "nconst", "inner")
print("joined table size: ", (data11.count(), len(data11.columns)))

df_title_basic_selected and df_title_principals_selected joined table size:  (53173247, 8)
joined table size:  (53163680, 12)
CPU times: user 1.02 s, sys: 87.7 ms, total: 1.11 s
Wall time: 6min 23s


In [21]:
%%time
data12 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data12.count(), len(data12.columns)))
data12 = data12.join(df_name_basics_selected, "nconst", "left")
print("joined table size: ", (data12.count(), len(data12.columns)))

df_title_basic_selected and df_title_principals_selected joined table size:  (53173247, 8)
joined table size:  (53173247, 12)
CPU times: user 1.05 s, sys: 79.7 ms, total: 1.13 s
Wall time: 6min 14s


In [22]:
%%time
data13 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "inner")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data13.count(), len(data13.columns)))
data13 = data13.join(df_name_basics_selected, "nconst", "inner")
print("joined table size: ", (data13.count(), len(data13.columns)))

df_title_principals_selected and df_title_basic_selected joined table size:  (53167105, 8)
joined table size:  (53157572, 12)
CPU times: user 1.12 s, sys: 79.5 ms, total: 1.2 s
Wall time: 6min 25s


In [23]:
%%time
data14 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "inner")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data14.count(), len(data14.columns)))
data14 = data14.join(df_name_basics_selected, "nconst", "left")
print("joined table size: ", (data14.count(), len(data14.columns)))

df_title_principals_selected and df_title_basic_selected joined table size:  (53167105, 8)
joined table size:  (53167105, 12)
CPU times: user 1.03 s, sys: 87.1 ms, total: 1.11 s
Wall time: 6min 15s


In [24]:
%%time
data15 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "left")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data15.count(), len(data15.columns)))
data15 = data15.join(df_name_basics_selected, "nconst", "inner")
print("joined table size: ", (data15.count(), len(data15.columns)))

df_title_principals_selected and df_title_basic_selected joined table size:  (53173247, 8)
joined table size:  (53163680, 12)
CPU times: user 1.25 s, sys: 122 ms, total: 1.37 s
Wall time: 6min 44s


In [25]:
%%time
data16 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "left")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data16.count(), len(data16.columns)))
data16 = data16.join(df_name_basics_selected, "nconst", "left")
print("joined table size: ", (data16.count(), len(data16.columns)))

df_title_principals_selected and df_title_basic_selected joined table size:  (53173247, 8)
joined table size:  (53173247, 12)
CPU times: user 1.11 s, sys: 113 ms, total: 1.22 s
Wall time: 6min 24s


Teraz zmierzymy czasy tych samych złączeń, jednak po usunięciu z tabel niepotrzebnych rekordów.

In [26]:
df_name_basics = df_name_basics.filter("primaryProfession like '%actor%' or primaryProfession like '%actress%'")
df_name_basics_selected = df_name_basics.select(["nconst", "primaryName", "birthYear", "deathYear", "knownForTitles"])
df_title_principals_selected = df_title_principals_selected.filter((df_title_principals_selected.category == "actor") | (df_title_principals_selected.category == "actress"))

In [27]:
%%time
data1 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data1.count(), len(data1.columns)))
data1 = data1.join(df_title_basic_selected, "tconst", "inner")
print("joined table size: ", (data1.count(), len(data1.columns)))

df_name_basics_selected and df_title_principals_selected joined table size:  (20190998, 8)
joined table size:  (20187521, 12)
CPU times: user 838 ms, sys: 59.1 ms, total: 897 ms
Wall time: 4min 51s


In [28]:
%%time
data2 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data2.count(), len(data2.columns)))
data2 = data2.join(df_title_basic_selected, "tconst", "left")
print("joined table size: ", (data2.count(), len(data2.columns)))

df_name_basics_selected and df_title_principals_selected joined table size:  (20190998, 8)
joined table size:  (20190998, 12)
CPU times: user 903 ms, sys: 92.7 ms, total: 996 ms
Wall time: 5min 2s


In [29]:
%%time
data3 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "right")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data3.count(), len(data3.columns)))
data3 = data3.join(df_title_basic_selected, "tconst", "inner")
print("joined table size: ", (data3.count(), len(data3.columns)))

df_name_basics_selected and df_title_principals_selected joined table size:  (20815083, 8)
joined table size:  (20811538, 12)
CPU times: user 827 ms, sys: 71.8 ms, total: 899 ms
Wall time: 4min 48s


In [30]:
%%time
data4 = df_name_basics_selected.join(df_title_principals_selected, "nconst", "right")
print("df_name_basics_selected and df_title_principals_selected joined table size: ", (data4.count(), len(data4.columns)))
data4 = data4.join(df_title_basic_selected, "tconst", "left")
print("joined table size: ", (data4.count(), len(data4.columns)))

df_name_basics_selected and df_title_principals_selected joined table size:  (20815083, 8)
joined table size:  (20815083, 12)
CPU times: user 803 ms, sys: 77 ms, total: 880 ms
Wall time: 4min 54s


In [31]:
%%time
data5 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "inner")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data5.count(), len(data5.columns)))
data5 = data5.join(df_title_basic_selected, "tconst", "inner")
print("joined table size: ", (data5.count(), len(data5.columns)))

df_title_principals_selected and df_name_basics_selected joined table size:  (20190998, 8)
joined table size:  (20187521, 12)
CPU times: user 825 ms, sys: 84.7 ms, total: 909 ms
Wall time: 4min 52s


In [32]:
%%time
data6 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "inner")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data6.count(), len(data6.columns)))
data6 = data6.join(df_title_basic_selected, "tconst", "left")
print("joined table size: ", (data6.count(), len(data6.columns)))

df_title_principals_selected and df_name_basics_selected joined table size:  (20190998, 8)
joined table size:  (20190998, 12)
CPU times: user 857 ms, sys: 101 ms, total: 958 ms
Wall time: 4min 54s


In [33]:
%%time
data7 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "left")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data7.count(), len(data7.columns)))
data7 = data7.join(df_title_basic_selected, "tconst", "inner")
print("joined table size: ", (data7.count(), len(data7.columns)))

df_title_principals_selected and df_name_basics_selected joined table size:  (20815083, 8)
joined table size:  (20811538, 12)
CPU times: user 834 ms, sys: 86 ms, total: 920 ms
Wall time: 4min 52s


In [34]:
%%time
data8 = df_title_principals_selected.join(df_name_basics_selected, "nconst", "left")
print("df_title_principals_selected and df_name_basics_selected joined table size: ", (data8.count(), len(data8.columns)))
data8 = data8.join(df_title_basic_selected, "tconst", "left")
print("joined table size: ", (data8.count(), len(data8.columns)))

df_title_principals_selected and df_name_basics_selected joined table size:  (20815083, 8)


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
%%time
data9 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "inner")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data9.count(), len(data9.columns)))
data9 = data9.join(df_name_basics_selected, "nconst", "inner")
print("joined table size: ", (data9.count(), len(data9.columns)))

In [ ]:
%%time
data10 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "inner")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data10.count(), len(data10.columns)))
data10 = data10.join(df_name_basics_selected, "nconst", "right")
print("joined table size: ", (data10.count(), len(data10.columns)))

In [ ]:
%%time
data11 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data11.count(), len(data11.columns)))
data11 = data11.join(df_name_basics_selected, "nconst", "inner")
print("joined table size: ", (data11.count(), len(data11.columns)))

In [ ]:
%%time
data12 = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("df_title_basic_selected and df_title_principals_selected joined table size: ", (data12.count(), len(data12.columns)))
data12 = data12.join(df_name_basics_selected, "nconst", "right")
print("joined table size: ", (data12.count(), len(data12.columns)))

In [ ]:
%%time
data13 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "inner")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data13.count(), len(data13.columns)))
data13 = data13.join(df_name_basics_selected, "nconst", "inner")
print("joined table size: ", (data13.count(), len(data13.columns)))

In [ ]:
%%time
data14 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "inner")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data14.count(), len(data14.columns)))
data14 = data14.join(df_name_basics_selected, "nconst", "right")
print("joined table size: ", (data14.count(), len(data14.columns)))

In [ ]:
%%time
data15 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "left")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data15.count(), len(data15.columns)))
data15 = data15.join(df_name_basics_selected, "nconst", "inner")
print("joined table size: ", (data15.count(), len(data15.columns)))

In [ ]:
%%time
data16 = df_title_principals_selected.join(df_title_basic_selected, "tconst", "left")
print("df_title_principals_selected and df_title_basic_selected joined table size: ", (data16.count(), len(data16.columns)))
data16 = data16.join(df_name_basics_selected, "nconst", "right")
print("joined table size: ", (data16.count(), len(data16.columns)))

Sprawdzimy jakie braki mają nasze dane.

In [ ]:
print(data11.filter(data11.primaryName.isNull()).count())
print(data11.filter(data11.category.isNull()).count())
print(data11.filter(data11.genres.isNull()).count())

0
0
3484


Po kilku uruchomieniach kodu, za każdym razem wychodzą różne wyniki, dlatego ciężko stwierdzić, który sposób łączenia danych jest najszybszy, jednak na pewno widać, że łączenie tabel bez zbędnych rekordów jest szybsze niż łączenie pełnych tabel. W ostatecznym rozrachunku, najlepiej wypada sposób łaczenia:
* df_title_basic_selected i df_title_principals_selected, typ łączenia "right"
* wcześniej złączone dane i df_name_basics_selected, typ łączenia "inner".

W danch tych nie ma żadnych nadmiarowych aktorów, pojawiają się jedynie braki co do filmów, gdyż nie wszystkie filmy, w których grali aktorzy pojawiają się w bazie.

Ten sposób łączenia danych zostanie zastosowany w notatniku głównym.


In [ ]:
print("df_name_basics_selected dataframe size: ", (df_name_basics_selected.count(), len(df_name_basics_selected.columns)))
print("df_title_principals_selected dataframe size: ", (df_title_principals_selected.count(), len(df_title_principals_selected.columns)))
print("df_title_basic_selected dataframe size: ", (df_title_basic_selected.count(), len(df_title_basic_selected.columns)))
data = df_title_basic_selected.join(df_title_principals_selected, "tconst", "right")
print("joined dataframe size: ", (data.count(), len(data.columns)))
data = data.join(df_name_basics_selected, "nconst", "inner")
print("joined dataframe size: ", (data.count(), len(data.columns)))

df_name_basics_selected dataframe size:  (4426315, 5)
df_title_principals_selected dataframe size:  (20798916, 4)
df_title_basic_selected dataframe size:  (9386465, 5)
joined dataframe size:  (20798916, 8)
joined dataframe size:  (20175420, 12)


# Wybierzmy przykładowe wiersze z końcowego dataframe'u

In [ ]:
sample = data.rdd.takeSample(False, 5)

In [ ]:
import pandas as pd
sample = pd.DataFrame(sample, columns = data.columns)
sample

,nconst,tconst,titleType,originalTitle,isAdult,genres,category,characters,primaryName,birthYear,deathYear,knownForTitles
0,nm0467502,tt1199515,tvMovie,Precek,0,Drama,actor,"[""Martin""]",Dragi Kostovski,1930,1980,"tt1247355,tt0182313,tt1369730,tt0182367"
1,nm0862117,tt1729191,tvMovie,Familienanschluß,0,Comedy,actor,"[""Gerichtsvollzieher""]",Hanno Thurau,1939,1992,"tt0108574,tt0080612,tt0806910,tt0131686"
2,nm0681566,tt0252262,short,Le bateau de Lu,0,Short,actor,"[""Un client""]",Michel Piccoli,1925,2020,"tt0118613,tt1456472,tt0057345,tt0061395"
3,nm5379363,tt16254088,tvEpisode,Episode #1.64,0,Drama,actress,"[""Ivie"",""Alero""]",Ivie Okujaye,\N,\N,"tt6911110,tt18182222,tt2526570,tt4452274"
4,nm4049315,tt4359146,tvEpisode,Episode #1.28,0,\N,actress,"[""Anastasia""]",Marina Shtoda,1982,\N,"tt6461726,tt2105514,tt4353482,tt12099306"
